In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm
import pandas as pd
import json

with open("/home/seongilpark/rag/GSM-IC_2step.json", "r") as f:
    df = pd.DataFrame(json.load(f))


In [13]:
len(list(set(df["original_question"].tolist())))

60

In [14]:
df["new_question"].nunique()

34140

In [9]:
relevant_template = 'Given the following text by a user, extract the part that is related and useful, so that using that text alone would be good context for providing an accurate and correct answer to the question portion of the text. Please include the actual question or query that the user is asking. Separate this into two categories labeled with ”Context text related to the question (includes all content except unrelated sentences):” and ”Detailed question:”.Do not use list. \n\nText by User: [ORIGINAL INPUT PROMPT]'
# [ORIGINAL INPUT PROMPT] : is the placeholder for the original input prompt. Please do not remove it.
unbiased_template = '[INPUT CONTEXT]\n\nAnswer in an unbiased way.'

In [6]:
def make_prompt(input_text):
    template = "<s>[INST] <<SYS>You are a helpful assitant.<</SYS>>>\n\n[INPUT] [/INST]"
    return template.replace("[INPUT]", input_text)

def S2A(question: str, tokenizer, model):
    s2a_prompt = make_prompt(relevant_template.replace("[ORIGINAL INPUT PROMPT]", question))
    print(s2a_prompt)
    encoded_prompt = tokenizer.encode(s2a_prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        first = model.generate(
        input_ids=encoded_prompt,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        )
    pred = tokenizer.decode(first[0], skip_special_tokens=True).split("[/INST]")[-1].strip()
    print(pred)
    real_prompt = make_prompt(unbiased_template.replace("[INPUT CONTEXT]", pred))
    print(real_prompt)
    encoded_prompt = tokenizer.encode(real_prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        second = model.generate(
        input_ids=encoded_prompt,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
        )
    real_pred = tokenizer.decode(second[0], skip_special_tokens=True).split("[/INST]")[-1].strip()
    return real_pred

In [4]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="meta-llama/Llama-2-13b-chat-hf",
                                             trust_remote_code=True,
                                             torch_dtype = torch.bfloat16,
                                             device_map="auto",
                                             use_flash_attention_2=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
from tqdm.auto import tqdm
preds, answers = [], []
for row_ in tqdm(sample_df.iterrows()):
    row = row_[1]
    q, a = row["new_question"], row["answer"]
    pred = S2A(q, tokenizer, model)
    preds.append(pred)
    answers.append(a)

0it [00:00, ?it/s]

<s>[INST] <<SYS>You are a helpful assitant.<</SYS>>>

Given the following text by a user, extract the part that is related and useful, so that using that text alone would be good context for providing an accurate and correct answer to the question portion of the text. Please include the actual question or query that the user is asking. Separate this into two categories labeled with ”Context text related to the question (includes all content except unrelated sentences):” and ”Detailed question:”.Do not use list. 

Text by User: Edmund owns a gift wrapping shop, he uses 18 inches of gift wrapper per gift box. Edmund's neighbor is 500 years old. If Edmund has 90 inches of gift wrapper per day, how many gift boxes will he be able to wrap every 3 days? [/INST]
Sure, I'd be happy to help! Here's the text you provided, broken down into two categories:

Context text related to the question (includes all content except unrelated sentences):

Edmund owns a gift wrapping shop, he uses 18 inches o

In [12]:
def cal_match_acc(answer: str, pred: str):
    answer = answer.lower().strip()
    pred = pred.lower().strip()
    if answer in pred:
        return True
    else:
        return False

'1a1d1b1a1s1d1 1'

In [13]:
for p, a in zip(preds, answers):
    acc = cal_match_acc(a, p)
    print(acc)

True
True
True
False
True
False
True
True
False
False
False
False
False
True
False
False
False
True
False
False


In [36]:
sample_df = df.sample(100)
questions, ctxs, answers = sample_df["question"].tolist(), sample_df["context"].tolist(), sample_df["answers"].tolist()

In [40]:
prompts, preds, is_answer = [], [], []
for q, c, a in zip(tqdm(questions), ctxs, answers):
    prompt = make_prompt(c, q)
    prompts.append(prompt)
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_new_tokens=200,
        )
    pred = tokenizer.decode(output_sequences[0], skip_special_tokens=True).split("[/INST]")[-1].strip()
    is_correct= any([ans.lower() in pred.lower() for ans in a])
    print(is_correct)
    is_answer.append(is_correct)
    preds.append(pred)

  0%|          | 0/100 [00:00<?, ?it/s]

True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [41]:
for a, p in zip(answers, preds):
    print("Answers:", a)
    print("Pred:", p)

Answers: {'text': ['(/ˈfrɛznoʊ/ FREZ-noh)', 'FREZ-noh', 'FREZ-noh'], 'answer_start': [7, 19, 19]}
Pred: Sure thing! Based on the context, the correct pronunciation of Fresno is /ˈfrɛznoʊ/ FREZ-noh.
Answers: {'text': ['Numerical models', 'Numerical', 'Numerical models'], 'answer_start': [448, 448, 448]}
Pred: Based on the context, numerical models of orogenic wedges are more sophisticated than analog models. Numerical models can include patterns of erosion and uplift, and can simulate the relationship between erosion and the shape of the mountain range, which is not possible with analog models. Additionally, numerical models can provide more detailed information about pathways for metamorphism through pressure, temperature, space, and time, making them more sophisticated.
Answers: {'text': ['mainline Protestant denomination', 'Protestant denomination', 'mainline Protestant denomination'], 'answer_start': [207, 216, 207]}
Pred: Sure! Based on the context, the answer to the question "In t

In [ ]:
output = tokenizer(make_prompt(q), return_tensors="pt", padding="longest").to(args["device"])
output = model.generate(**output, max_new_tokens=args["max_tokens"])
pred = tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [16]:
for i, row in enumerate(dataset["validation"]):
    if "-" in row["id"]:
        print(row)
        if i > 20:
            break   

{'id': '56be4db0acb8001400a502ee-high-conf-turk1', 'title': 'Super_Bowl_50', 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Champ Bowl 40 took place in Chicago.', 'question': 'Where did Super Bowl 50 take place?', 'answers': {'text': ['Santa Clara, Californi

model = 

In [23]:
df

,validation
0,"{'id': '56be4db0acb8001400a502ee', 'title': 'S..."
1,"{'id': '56bea9923aeaaa14008c91bb', 'title': 'S..."
2,"{'id': '56beace93aeaaa14008c91e0', 'title': 'S..."
3,"{'id': '56bf10f43aeaaa14008c94fd', 'title': 'S..."
4,"{'id': '56bf10f43aeaaa14008c9500', 'title': 'S..."
...,...
1782,"{'id': '57378c9b1c456719005744a7', 'title': 'F..."
1783,"{'id': '57379ed81c456719005744d6', 'title': 'F..."
1784,"{'id': '5737a9afc3c5551400e51f64', 'title': 'F..."
1785,{'id': '5737a9afc3c5551400e51f64-high-conf-tur...
